<a href="https://colab.research.google.com/github/ShykerBogdan/Prozzoro_project/blob/master/Data_prepr_batches_100_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import time
from datetime import datetime
import numpy as np

In [3]:
filename='/content/gdrive/My Drive/ML_proz/data_source_cleaned.csv'
df=pd.read_csv(filename)
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,14,15,16,17,18,24,38,48,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3507778, 57)

In [0]:
df['tenders.procuringEntity_address_postalCode']=df['tenders.procuringEntity_address_postalCode'].astype(str).str[:2]
table_postal_code={'Київ':['01', '02', '03', '04', '05', '06'],
                  'Київська область':['07', '08', '09'],
                  'Житомирська область':['10', '11', '12', '13'],
                  'Черніговська область':['14', '15', '16', '17'],
                  'Черкаська область':['18', '19', '20'],
                  'Вінницька область':['21', '22', '23', '24'],
                  'Кіровоградська область':['25', '26', '27', '28'],
                  'Хмельницька область':['29', '30', '31', '32'],
                  'Рівненська область': ['33', '34', '35'],
                  'Полтавська область':['36', '37', '38', '39'],
                  'Сумська область':['40', '41', '42'],
                  'Волинська область':['43', '44', '45'],
                  'Тернопольська область':['46', '47', '48'],
                  'Дніпропетровська область':['49', '50', '51', '52', '53'],
                  'Миколаївська область':['54', '55', '56', '57'],
                  'Чернівецька область':['58', '59', '60'],
                  'Харьківська область':['61', '62', '63', '64'],
                  'Одеська область':['65', '66', '67', '68'],
                  'Запоріжська область':['69', '70', '71', '72'],
                  'Херсонська область':['73', '74', '75'],
                  'Івано-Франківська область':['76', '77', '78'],
                  'Львівська область':['79', '80', '81', '82'],
                  'Донецька область':['83', '84', '85', '86', '87'],
                  'Закарпатська область':['88', '89', '90'],
                  'Луганська область':['91', '92', '93', '94'],
                  'Автономна республіка Крим':['95', '96', '97', '98'],
                  'Севастопіль':['99']}
regions=dict()
for key,value in table_postal_code.items():
    for item in value:
        regions[item]=key
def get_region(raw,regions):
    try:
        raw['region']=regions[raw['tenders.procuringEntity_address_postalCode']]
    except:  
        raw['region']=np.nan
    return raw
df=df.apply(get_region,args=(regions,),axis=1)

In [0]:
class DataPreproccessing:
    def __init__(self, data_source, churn_interval=90, start_period=None, end_period=None,start_data_unique=None):
        if start_period and end_period:
          data_source=data_source[pd.to_datetime(data_source['new_date'])>=start_period]
          data_source=data_source[pd.to_datetime(data_source['new_date'])<end_period]
        self.start_period=start_period
        self.end_period=end_period
        self.start_data_unique=start_data_unique
        # сформували 2 вікна(X,y)
        self.last_day = pd.to_datetime('2018-11-29')
        # для статистики постачальника вибираємо проміжок [start_period,end_period]
        self.divide_data = pd.to_datetime(self.end_period) - pd.DateOffset(days=churn_interval)
        print(self.divide_data)
        self.data_source = data_source[pd.to_datetime(data_source['new_date']) <= self.divide_data]
        
        # для labeling вибираємо період [end_period, end_period+churn_interval]
        self.label_data = data_source[pd.to_datetime(data_source['new_date']) > self.divide_data]
        # вибираємо унікальних постачальників
#         self.unique_id_data=self.data_source[(pd.to_datetime(self.data_source['new_date'])>=self.start_data_unique) & (pd.to_datetime(self.data_source['new_date'])<=self.divide_data)]
        self.unique_data=self.data_source[pd.to_datetime(self.data_source['new_date'])>=self.start_data_unique]
        self.unique_data=self.unique_data[pd.to_datetime(self.unique_data['new_date'])<=self.divide_data]
        self.unique_id = list(self.unique_data['participants'].value_counts().index)
        # створюємо dataframe з колонкою унікальних постачальників і туди будемо додавати features
        self.feature_data = pd.DataFrame(self.unique_id, columns=['unique_id'])
        print(self.feature_data[self.feature_data['unique_id']=='2969913153'])    
    
    def print_dates(self):
        print('Features: {0}/{1}'.format(self.start_period,self.divide_data))
        print('Unique_id: {0}/{1}'.format(self.start_data_unique,self.divide_data))
        print('Labeling: {0}/{1}'.format(self.divide_data,self.end_period))
    
#     def activity_date(self):
#         f = ['winner', 'tenders.auctionPeriod_startDate', 'contracts.dateSigned', 'date_from_id', 'awards.date']
#         self.data_source['new_date'] = self.data_source[f].apply(
#             lambda x: x[f[2]] if x[f[0] == 1] else (x[f[3]] if pd.isnull(x[f[1]]) else x[f[1]]), axis=1)
#         self.data_source['new_date'] = self.data_source[['new_date']].apply(lambda x: x['new_date'][:10], axis=1)
#         self.data_source['new_date'] = self.data_source[['new_date', 'date_from_id']].apply(
#             lambda x: x['date_from_id'] if x['new_date'] == '\\N' else x['new_date'], axis=1)

    def cleaning(self):
#         self.data_source['winner'] = self.data_source[['contracts.value_amount', 'contracts.status', 'winner']].apply \
#             (lambda x: 0 if x['contracts.value_amount'] == '\\N'
#                             or x['contracts.status'] == 'cancelled' else x['winner'], axis=1)
#         print('winner updated')
#         self.activity_date()
#         print('activity date')

#         def convert_to_float(raw):
#             try:
#                 raw['lots.value_amount'] = float(raw['lots.value_amount'])
#             except:
#                 raw['lots.value_amount'] = np.nan
#             return raw

#         self.data_source['lots.value_amount'] = self.data_source[['lots.value_amount']].apply(convert_to_float, axis=1)
#         print('convert lots')
        self.data_source['tenders.procuringEntity_address_postalCode']=self.data_source['tenders.procuringEntity_address_postalCode'].astype(str).str[:2]
        table_postal_code={'Київ':['01', '02', '03', '04', '05', '06'],
                  'Київська область':['07', '08', '09'],
                  'Житомирська область':['10', '11', '12', '13'],
                  'Черніговська область':['14', '15', '16', '17'],
                  'Черкаська область':['18', '19', '20'],
                  'Вінницька область':['21', '22', '23', '24'],
                  'Кіровоградська область':['25', '26', '27', '28'],
                  'Хмельницька область':['29', '30', '31', '32'],
                  'Рівненська область': ['33', '34', '35'],
                  'Полтавська область':['36', '37', '38', '39'],
                  'Сумська область':['40', '41', '42'],
                  'Волинська область':['43', '44', '45'],
                  'Тернопольська область':['46', '47', '48'],
                  'Дніпропетровська область':['49', '50', '51', '52', '53'],
                  'Миколаївська область':['54', '55', '56', '57'],
                  'Чернівецька область':['58', '59', '60'],
                  'Харьківська область':['61', '62', '63', '64'],
                  'Одеська область':['65', '66', '67', '68'],
                  'Запоріжська область':['69', '70', '71', '72'],
                  'Херсонська область':['73', '74', '75'],
                  'Івано-Франківська область':['76', '77', '78'],
                  'Львівська область':['79', '80', '81', '82'],
                  'Донецька область':['83', '84', '85', '86', '87'],
                  'Закарпатська область':['88', '89', '90'],
                  'Луганська область':['91', '92', '93', '94'],
                  'Автономна республіка Крим':['95', '96', '97', '98'],
                  'Севастопіль':['99']}
        regions=dict()
        for key,value in table_postal_code.items():
            for item in value:
              regions[item]=key
        def get_region(raw,regions):
            
            try:
              raw['region']=regions[raw['tenders.procuringEntity_address_postalCode']]
            except:  
              raw['region']=np.nan
            return raw
        self.data_source=self.data_source.apply(get_region,args=(regions,),axis=1)
        print('regions updated')
        
    def split_into_batches(self, n=20000, filename=None):
        
        self.cleaning()
        feature_batches = list()
        n_iter = int(np.ceil(self.feature_data.shape[0] / n))
        for i in range(n_iter):
            start_time = time.time()
            print('Batch {0}-{1}:'.format(i * n, (i + 1) * n))
            unique_id = self.feature_data.iloc[i * n:(i + 1) * n]['unique_id'].values
            feature_df = pd.DataFrame(unique_id, columns=['unique_id'])
            feature_batches.append(feature_df)
            # part of data_source
            df_ = self.data_source.loc[self.data_source['participants'].isin(unique_id)]
            print('Unique_supp:{0}-{1}'.format(len(unique_id), df_.shape[0]))
            df_ = df_.sort_values(by='new_date')
            self.divide_data = str(self.divide_data)[:10]

            # features:
            # 0. First activity
            # 1. Last activity
            def get_last_activity(raw, data,eps=0.7):
                date=data[data['participants'] == raw['unique_id']]
                dates=date['new_date'].values
                # favourite host
                try:
                  if date['tenders.procuringEntity_identifier_id'].value_counts().iloc[0]/date.shape[0] >=eps:
                    raw['favourite_host']=1
                  else:
                    raw['favourite_host']=0
                  raw['count_unique_hosts']=date['tenders.procuringEntity_identifier_id'].value_counts().shape[0]
                except:
                  raw['favourite_host']=np.nan
                  raw['count_unique_hosts']=np.nan
                
                
#                 raw['unique_regions']=len(set(date['region'].values))
#                 raw['last_activity_date'] = dates[-1]
                try:
                    raw['first_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(dates[0])).days
                    raw['last_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(dates[-1])).days
                    if len(dates)!=1:
                      raw['average_activity']=(pd.to_datetime(dates[-1])-pd.to_datetime(dates[0])).days/(len(dates)-1)
                    else:
                      raw['average_activity']='?'
                except:
                    print('--------------')
                    raw['average_activity'] = '?'
                    
                return raw

            feature_batches[i] = feature_batches[i].apply(get_last_activity, args=(df_[['participants','new_date','tenders.procuringEntity_identifier_id','tenders.id']],0.75,),
                                                          axis=1)
            feature_batches[i]['last_activity_days'] = feature_batches[i].apply(
                lambda x: 0 if x['last_activity_days'] < 0 else x['last_activity_days'], axis=1)
            print(" --- %s seconds ---" % (time.time() - start_time))

            # 2. Average activity
#             def get_average_activity(raw, data):
#                 dates_ = list(data[data['participants'] == raw['unique_id']]['new_date'].values)
#                 try:
#                     raw['average_activity'] = (pd.to_datetime(dates_[-1]) - pd.to_datetime(dates_[0])).days / len(dates_)
#                 except:
#                     raw['average_activity'] = np.nan
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(get_average_activity, args=(df_[['participants','new_date']],),
#                                                           axis=1)

            # 3. Count lots
            freq = df_['participants'].value_counts()

            def count_lots(raw, frequency):
                raw['count_lots'] = frequency[raw['unique_id']]
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lots, args=(freq,), axis=1)

            # 4. Win, lose
            frequency = df_[df_['winner'] == 1]['participants'].value_counts()
            print(" --- %s seconds ---" % (time.time() - start_time))
            def win_lose(raw, frequency):
                # use try bsc if index not in frequency 'win'=0
                try:
                    raw['win'] = frequency[raw['unique_id']]
                except:
                    raw['win'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(win_lose, args=(
                frequency,), axis=1)
            feature_batches[i]['lose'] = feature_batches[i]['count_lots'] - feature_batches[i]['win']
            print(" --- %s seconds ---" % (time.time() - start_time))
#             5. Count win open

            frequency = df_.loc[(df_['winner'] == 1) & (df_['tenders.procurementMethod'] == 'open')][
                'participants'].value_counts()

            def count_win_open(raw, frequency):
                try:
                    raw['win_open'] = frequency[raw['unique_id']]
                except:
                    raw['win_open'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(count_win_open, args=(
                frequency,), axis=1)
            feature_batches[i]['win_not_open'] = feature_batches[i]['win'] - feature_batches[i]['win_open']
            print(" --- %s seconds ---" % (time.time() - start_time))
#             6. Count lose open
            frequency = df_.loc[(df_['winner'] == 0) & (df_['tenders.procurementMethod'] == 'open')][
                'participants'].value_counts()

            def count_lose_open(raw, frequency):
                try:
                    raw['lose_open'] = frequency[raw['unique_id']]
                except:
                    raw['lose_open'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lose_open, args=(
                frequency,), axis=1)
#             feature_batches[i]['lose_not_open'] = feature_batches[i]['lose'] - feature_batches[i]['lose_open']
            print(" --- %s seconds ---" % (time.time() - start_time))
            # 7. Average economy
            data = df_[df_['winner'] == 1][
                ['participants', 'awards.value_amount', 'lots.value_amount']]
            data['economy_value'] = data['lots.value_amount'] - data['awards.value_amount']
            data['economy_percent'] = data['awards.value_amount'] / data['lots.value_amount']

            def get_economy(raw, data):
                d = data[data['participants'] == raw['unique_id']]
                raw['economy_value'] = np.nanmean(d['economy_value'])
                raw['average_winner_price']=np.nansum(d['awards.value_amount'])/d.shape[0]
                raw['economy_percent'] = np.nanmean(d['economy_percent'])
                return raw

            feature_batches[i] = feature_batches[i].apply(get_economy, args=(data,), axis=1)
#             8. Get label
            participants = list(self.label_data['participants'].value_counts().index)

            def get_y(raw, participants):
              if raw['unique_id'] in participants:
                  raw['y'] = 1
              else:
                  raw['y'] = 0
              return raw
#             9. Get favourite hosts
            feature_batches[i] = feature_batches[i].apply(get_y, args=(participants,), axis=1)
#             def favourite_percent(raw, data, eps = 0.7):
#                 d = data[data['participants']==raw['unique_id']].drop_duplicates('tenders.id')
#                 try:
#                   if d['tenders.procuringEntity_identifier_id'].value_counts().iloc[0]/d.shape[0] >=eps:
#                     raw['favourite_host']=1
#                   else:
#                     raw['favourite_host']=0
#                     raw['count_unique_hosts']=d['tenders.procuringEntity_identifier_id'].value_counts().shape[0]
#                 except:
#                   raw['favourite_host']=np.nan
#                   raw['count_unique_hosts']=np.nan
#                 del d
#                 return raw
#             feature_batches[i] = feature_batches[i].apply(favourite_percent, args=(df_[['participants', 'tenders.id', 'tenders.procuringEntity_identifier_id']],0.75,), axis=1)
            # 10. Get unique region
            print(" Batch %s seconds ---" % (time.time() - start_time))
        if filename:
            print(f'write to csv')
            feature_batches[i].to_csv(filename + '{0}'.format(i) + '.csv', index=False)

        
        # concat dataframes into one
        print('Concat')
        all_batches_df = pd.concat(feature_batches)
        self.feature_data = all_batches_df.copy()
        if filename:
            print('write to csv_all')
            all_batches_df.to_csv(filename + '_all.csv', index=False)
        print(" --- %s seconds ---" % (time.time() - start_time))
    def get_label(self):
        participants = list(self.label_data['participants'].value_counts().index)

        def get_y(raw, participants):
            if raw['unique_id'] in participants:
                raw['y'] = 1
            else:
                raw['y'] = 0
            return raw

        self.feature_data = self.feature_data.apply(get_y, args=(participants,), axis=1)


In [0]:
col=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned',
     'awards.date','participants','date_from_id','contracts.value_amount','tenders.procurementMethod','contracts.status','awards.value_amount','tenders.id','tenders.procuringEntity_identifier_id','lots.value_amount']

In [0]:
# start_time_all = time.time()
# df1=df[col]
# prepr = DataPreproccessing(df1,108,start_period='2016-08-09')
# prepr.split_into_batches(n=10000,filename='model')
# print("--- %s seconds ---" % (time.time() - start_time_all))
# print(prepr.feature_data.head())

2018-08-13 00:00:00
winner updated
activity date
convert lots
Batch 0-10000:
Unique_supp:10000-1621413
 --- 3430.168293952942 seconds ---
 --- 3439.3191719055176 seconds ---
 --- 3446.271993160248 seconds ---
 --- 3453.78662443161 seconds ---
 --- 3461.5636405944824 seconds ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:218: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:219: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:220: RuntimeWarning: Mean of empty slice


 Batch 5536.0409734249115 seconds ---
Batch 10000-20000:
Unique_supp:10000-351882
 --- 864.6975250244141 seconds ---
 --- 872.2511448860168 seconds ---
 --- 879.3677442073822 seconds ---
 --- 886.6838326454163 seconds ---
 --- 894.0160582065582 seconds ---
 Batch 1509.958750486374 seconds ---
Batch 20000-30000:
Unique_supp:10000-207031
 --- 530.7177498340607 seconds ---
 --- 538.1015701293945 seconds ---
 --- 545.1780285835266 seconds ---
 --- 552.4168667793274 seconds ---
 --- 559.7113778591156 seconds ---


In [37]:
prepr = DataPreproccessing(df.iloc[:100000],start_period='2016-08-01',end_period='2017-04-01',start_data_unique='2016-10-04',churn_interval=90)
prepr.print_dates()
prepr.split_into_batches(n=10000)
print(prepr.feature_data.head(10))

2017-01-01 00:00:00
       unique_id
1532  2969913153
Features: 2016-08-01/2017-01-01 00:00:00
Unique_id: 2016-10-04/2017-01-01 00:00:00
Labeling: 2017-01-01 00:00:00/2017-04-01
regions updated
Batch 0-10000:
Unique_supp:5405-8852
--------------


KeyboardInterrupt: ignored

In [30]:
prepr.feature_data[prepr.feature_data['unique_id']=='2969913153']

,unique_id,favourite_host,count_unique_hosts,first_activity_days,last_activity_days,average_activity,count_lots,win,lose,win_open,win_not_open,lose_open,economy_value,average_winner_price,economy_percent,y
1177,2969913153,0,10,136,89,5.22222,10,3,7,3,0,7,264.0,8628.0,0.953115,0


In [31]:
print(prepr.feature_data.head(100))

     unique_id  favourite_host  count_unique_hosts  first_activity_days  \
0     25394112               0                  24                   53   
1     37149148               0                  23                  125   
2     32490244               0                  19                   59   
3     39726851               0                  17                   16   
4     31816235               0                  15                   46   
5     38385636               0                  12                   41   
6   2969913153               0                  10                   45   
7     33614922               0                   8                   34   
8     23522853               0                   7                   61   
9     23291827               0                   7                   34   
10    04947995               0                   7                   47   
11    38516299               0                   6                   27   
12    38703629           

In [12]:
prepr..head(100)

,unique_id
0,25394112
1,32490244
2,31816235
3,37149148
4,39726851
5,2969913153
6,39190161
7,33614922
8,38385636
9,04947995
